# Outlier Detection

This notebook will walk you through a workflow for outlier detection.

## TOC

 * [Detecting outlier jobs](#detect-outlier-jobs-study-1)
  * [partition jobs](#partition-jobs)
  * [create a trained model](#create-ref-model)
 * [Root Cause Analysis (RCA)](#rca-job)
 * [Detecting outlier operations](#detect-outlier-ops)
  * [RCA for operations](#rca-ops)
 * [Notes](#notes)

## <a name="detect-outlier-jobs-study-1">Detecting outlier jobs</a>

In this example we will use synthetic data. The data was generated using:
```
sample/outlier/workload.sh
```

The script basically compiles the linux kernel a few times. For certain compiles
it adds a background workload, so the compile takes longer. The slower job is marked
with the an 'outlier' suffix. However, we will pretend we don't know the outlier
and figure it out.

Along the way, we will also learn how to create a trained model, and use it 
for future outlier detection.


### Requirements

You will need to import data:
```
$ EPMT_TZ=Asia/Kolkata ./epmt -v submit test/data/outliers_nb/kernel/*.tgz
```

In [1]:
import epmt_query as eq
import epmt_outliers as eod

In [2]:
jobs = eq.get_jobs(tags='exp_name:linux_kernel', fmt='terse')
jobs

['kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-192044-outlier',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819',
 'kern-6656-20190614-201744-outlier']

In [3]:
# As a first pass let's see whether the outliers can be auto-detected
(df, fdict) = eod.detect_outlier_jobs(jobs)
df

,jobid,duration,num_procs,cpu_time
0,kern-6656-20190614-185359,0,0,1
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,0,1
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,0,1


As you can see, while we did catch both the outliers, there
is also the "false positive" on one "non-outlier" process
The reason the 1 is marked for `duration` and `cpu_time` but `not num_procs`
is because the background compute process increased the job duration
but not the number of sub-processes of our workload.

### <a name="partition-jobs">Partitioning the jobs set into two sets</a>

`fdict`, obtained above, is a dictionary that contains for each feature 
(`cpu_time`, `duration` or `num_procs`), two sets, the first is a set of 
"reference" jobs and the second is a set of "outlier" jobs as detected by 
that particular feature.

In [4]:
fdict

{'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'}),
 'num_procs': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819',
   'kern-6656-20190614-201744-outlier'},
  set()),
 'cpu_time': ({'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-185359',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-201744-outlier'})}

Partitioning can be more simply obtained using the following call:

In [5]:
parts = eod.partition_jobs(jobs, features=['duration'])
parts

{'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'})}

Above, we just got the partitioning of the jobs on a single `feature` -- `duration`.
You could also specify multiple features, such as `['duration', 'cpu_time']`.

### <a name="create-ref-model">Creating a trained (reference) model</a>

If we have a set of reference jobs, we can create a "trained" model. Subsequently we can do
outlier detection using the trained model. Let's use the reference set we obtained above:

In [6]:
ref_jobs = parts['duration'][0]
ref_jobs

{'kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819'}

When creating the trained model, we can specify a `tag` of our choosing. This `tag` can
be used later to retrieve the trained model from the database.

In [7]:
r = eq.create_refmodel(ref_jobs, tag='exp_name:linux_kernel;type:ref', name='my_model')

In [8]:
r['id'], r['tags'], r['name']

(1, {'exp_name': 'linux_kernel', 'type': 'ref'}, 'my_model')

In [9]:
# using the trained model is as simple as:
(df, _) = eod.detect_outlier_jobs(jobs, trained_model = r['id'])
df

,jobid,duration,num_procs,cpu_time
0,kern-6656-20190614-185359,0,0,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,0,1
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,0,1


Obviously the jobs that were used to create the reference model will not be 
classifed as outliers for any feature.

## <a name="rca-job">Root Cause Analysis (RCA)</a>

In this study we will do an RCA with real data generated from GFDL PP runs.


### Requirements
Please import the following data:

```
$ ./epmt -v submit test/data/outliers_nb/*.tgz
```

All these jobs share the following tags: `{u'ocn_res': u'0.5l75', u'atm_res': u'c96l49', u'exp_component': u'ocean_annual_z_1x1deg', u'exp_name': u'ESM4_historical_D151'}`. The difference is only that they have different values for `('exp_time', 'script_name')`.
<div></div>

<!--
<details>
  <summary>Advanced query</summary>
  If you are curious how we found these comparable jobs, here is the query (ADVANCED TOPIC):
  ```
>>> x = eq.Job.select(lambda j: j.tags['exp_component'] == 'ocean_annual_z_1x1deg').filter(lambda j: j.tags['exp_name'] == 'ESM4_historical_D151')
>>> eq.get_jobs(x, fmt="terse")
  ```
</details>
-->

In [10]:
jobs = eq.get_jobs(tags="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')
jobs

['625151',
 '627907',
 '629322',
 '633114',
 '675992',
 '680163',
 '685001',
 '691209',
 '693129']

Now, let's partition the jobs by `cpu_time`: 

In [11]:
parts = eod.partition_jobs(jobs, features=['cpu_time'])
parts

{'cpu_time': ({'627907',
   '629322',
   '633114',
   '675992',
   '680163',
   '685001',
   '691209',
   '693129'},
  {'625151'})}

As you can see, the first partition contains conformant jobs, and the outlier
partition contains a single job `625151`. Let's see how different is the `cpu_time`
of this job v. the rest:

In [13]:
jobs_df = eq.get_jobs(jobs, fmt='pandas', order=(eq.desc(eq.Job.cpu_time)))
display(jobs_df.columns.values)
jobs_df[['jobid', 'cpu_time', 'duration', 'num_procs']]

array(['start', 'jobname', 'created_at', 'end', 'exitcode', 'duration',
       'updated_at', 'tags', 'info_dict', 'env_dict', 'cpu_time',
       'annotations', 'env_changes_dict', 'analyses', 'submit', 'jobid',
       'user', 'all_proc_tags', 'num_procs', 'num_threads', 'systemtime',
       'rssmax', 'syscw', 'invol_ctxsw', 'time_oncpu', 'inblock',
       'user+system', 'usertime', 'time_waiting', 'outblock', 'rchar',
       'syscr', 'rdtsc_duration', 'PERF_COUNT_SW_CPU_CLOCK',
       'write_bytes', 'processor', 'vol_ctxsw', 'guest_time', 'majflt',
       'cancelled_write_bytes', 'timeslices', 'read_bytes', 'wchar',
       'delayacct_blkio_time', 'minflt'], dtype=object)

,jobid,cpu_time,duration,num_procs
0,625151,1.224445e+09,1.042562e+10,13530
1,627907,6.945949e+08,6.589175e+09,4411
2,629322,6.221380e+08,7.286332e+09,4411
3,691209,5.366800e+08,8.601632e+08,4411
4,693129,5.336865e+08,3.619325e+09,4411
5,675992,5.208372e+08,9.114151e+09,4411
6,685001,4.803105e+08,6.815710e+09,4428
7,680163,4.629361e+08,6.156192e+09,4411
8,633114,4.571809e+08,6.036720e+09,4445


It's really interesting that the outlier has thrice the number of processes
as the others. Let's see if we can uncover more..

In [14]:
(refs, outl) = parts['cpu_time']
(refs, outl)

({'627907',
  '629322',
  '633114',
  '675992',
  '680163',
  '685001',
  '691209',
  '693129'},
 {'625151'})

In [15]:
(_, df, flist) = eod.detect_rootcause(refs, '625151')
df

,num_procs,cpu_time,duration
count,8.000000,8.000000e+00,8.000000e+00
mean,4417.375000,5.385455e+08,5.809721e+09
std,12.648405,8.229571e+07,2.512447e+09
min,4411.000000,4.571809e+08,8.601632e+08
25%,4411.000000,4.759669e+08,5.432371e+09
50%,4411.000000,5.272618e+08,6.372683e+09
75%,4415.250000,5.580445e+08,6.933366e+09
max,4445.000000,6.945949e+08,9.114151e+09
input,13530.000000,1.224445e+09,1.042562e+10
ref_max_modified_z_score,3.597300,2.028600e+00,5.481300e+00


The features are ranked from the one with the highest `modified_z_score_ratio` from left to right, 
in decreasing importance.

In [16]:
flist

[('num_procs', 268.2083785061018),
 ('cpu_time', 4.166370896184561),
 ('duration', 0.735227044679182)]

If you would like to expand the RCA to other `features`, do as below:

In [17]:
(_, df, flist) = eod.detect_rootcause(refs, '625151', features=[])
df

,wchar,syscw,rchar,num_threads,num_procs,rssmax,syscr,vol_ctxsw,timeslices,minflt,...,invol_ctxsw,inblock,read_bytes,cancelled_write_bytes,majflt,rdtsc_duration,exitcode,processor,guest_time,delayacct_blkio_time
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,...,8.000000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.0,8.0,8.0,8.0
mean,7.272861e+10,4.052586e+06,9.568890e+10,4682.750000,4417.375000,6.962899e+07,1.220181e+07,8.234889e+05,8.506098e+05,7.527056e+06,...,22424.625000,1.890426e+07,9.678984e+09,2.056820e+09,558.375000,-1.012455e+16,0.0,0.0,0.0,0.0
std,8.307445e+04,3.171187e+02,2.773749e+06,13.392429,12.648405,2.277324e+05,5.538603e+03,2.070822e+04,2.889237e+04,2.128115e+05,...,21975.570631,2.812171e+07,1.439832e+10,5.042457e+08,1298.907226,2.000890e+16,0.0,0.0,0.0,0.0
min,7.272851e+10,4.052375e+06,9.568720e+10,4676.000000,4411.000000,6.927494e+07,1.219834e+07,8.020240e+05,8.203150e+05,7.328295e+06,...,12130.000000,6.736000e+03,3.448832e+06,8.088945e+08,2.000000,-5.328561e+16,0.0,0.0,0.0,0.0
25%,7.272854e+10,4.052453e+06,9.568735e+10,4676.000000,4411.000000,6.957019e+07,1.219881e+07,8.118298e+05,8.298062e+05,7.372435e+06,...,13439.750000,2.827020e+05,1.447434e+08,2.234230e+09,11.750000,-6.987199e+15,0.0,0.0,0.0,0.0
50%,7.272860e+10,4.052482e+06,9.568751e+10,4676.000000,4411.000000,6.962996e+07,1.219916e+07,8.137210e+05,8.383915e+05,7.527902e+06,...,14215.500000,3.867224e+06,1.980019e+09,2.236223e+09,45.000000,3.984634e+13,0.0,0.0,0.0,0.0
75%,7.272867e+10,4.052529e+06,9.568891e+10,4680.500000,4415.250000,6.965483e+07,1.220205e+07,8.321982e+05,8.713520e+05,7.543403e+06,...,16518.500000,2.853633e+07,1.461061e+10,2.236244e+09,220.500000,5.426578e+13,0.0,0.0,0.0,0.0
max,7.272873e+10,4.053353e+06,9.569491e+10,4712.000000,4445.000000,7.007920e+07,1.221461e+07,8.624530e+05,8.950970e+05,8.003186e+06,...,76421.000000,7.813116e+07,4.000315e+10,2.236248e+09,3750.000000,7.181059e+13,0.0,0.0,0.0,0.0
input,1.388007e+11,4.436801e+06,9.854065e+10,14551.000000,13530.000000,1.336608e+08,1.308468e+07,3.150991e+06,3.198404e+06,2.055072e+07,...,32844.000000,2.502162e+07,1.281107e+10,2.331615e+09,80.000000,1.009561e+14,0.0,0.0,0.0,0.0
ref_max_modified_z_score,1.223000e+00,1.397980e+01,2.127270e+01,3.597300,3.597300,5.644000e+00,1.772740e+01,4.232200e+00,2.765000e+00,3.701900e+00,...,38.777800,1.298700e+01,1.298700e+01,4.700847e+04,62.475600,1.580507e+03,0.0,0.0,0.0,0.0


Note that the metrics are aggregates across the underlying threads/processes across a job.

In our next study we will attempt to determine which `operations` from within the jobs were outliers.

## <a name="detect-outlier-ops">Detect Outlier Operations</a>

This study shows how we can find outlier operations across a set of jobs.

Please [review the previous study](#rca-job) as this has the same requirements.

In [18]:
jobs = eq.get_jobs(tags="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')

In [19]:
len(jobs)

9

In [20]:
# widen width of column display width to show full tag
import pandas as pd
pd.set_option('display.max_colwidth', 200)
(df, parts, scores_df, sorted_tags, sorted_features) = eod.detect_outlier_ops(jobs)
df.head(20)

,jobid,tags,duration,cpu_time,num_procs
0,625151,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
1,627907,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
2,629322,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
3,633114,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
4,675992,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",1,0,0
5,680163,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
6,685001,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
7,691209,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
8,693129,"{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'}",0,0,0
9,625151,"{'op': 'hsmget', 'op_instance': '7', 'op_sequence': '19'}",0,0,0


In [21]:
len(sorted_tags)

416

In [22]:
sorted_tags[:5]

[{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'},
 {'op': 'hsmget', 'op_instance': '7', 'op_sequence': '19'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '12'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '21'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '15'}]

As you can see there are 416 unique operations. Since we want to focus our attention
on the operations, that have the highest deviation across jobs, `detect_outlier_ops`
helps us by ordering the dataframe, `df` in the order of decreasing operation (tag)
importance. To figure out the importance of a tag, it uses the maximum of the scores
across all jobs, across all features for that particular tag. `scores_df` and `sorted_tags`
are similarly ordered by decreasing tag importance.

In [23]:
scores_df.head()

,tags,duration,cpu_time,num_procs
0,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""18""}",776.463,0.000,0.0
1,"{""op"": ""hsmget"", ""op_instance"": ""7"", ""op_sequence"": ""19""}",725.382,3.166,0.0
2,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""12""}",368.382,0.000,0.0
3,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""21""}",293.405,0.026,0.0
4,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""15""}",230.617,0.000,0.0


`detect_outlier_ops` thus already helps with RCA by ordering the output
in decreasing tag importance. It goes even further, by presenting a ordered
`features` list, in descreasing order of feature importance. Here the
importance of a `feature` is determined by summing the scores of the
feature across all tags. 

In [24]:
sorted_features

['duration', 'cpu_time', 'num_procs']

<a name="rca-ops"></a>Once you have the ordered list of tags (operations), you may want to do
further RCA analysis for a specific operation. For RCA analysis at the operation-level, 
we specify a *reference set* of jobs and an outlier job (similar to how we did for RCA at the job-level). 
In addition, we specify the operation of interest. The goal of the op-RCA is to rank the 
features in order of importance.

Let's suppose we care about the top operation: `{"op": "ncatted", "op_instance": "3", "op_sequence": "32"}`.
We see the jobid `629322` was one of the outliers for this operation in `df` for both the `duration` and
`cpu_time` features.

In [25]:
# first derive the list of jobs other than the outlier
refjobs = eq.get_jobs(tags="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fltr=(eq.Job.jobid != "629322"), fmt='terse')
refjobs

['625151',
 '627907',
 '633114',
 '675992',
 '680163',
 '685001',
 '691209',
 '693129']

In [26]:
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(refjobs, '629322', {"op": "ncatted", "op_instance": "3", "op_sequence": "32"})
ret

True

In [27]:
df_rca

,duration,cpu_time,num_procs
count,7.000000,7.000000,7.0
mean,1188.428571,24566.571429,1.0
std,114.402589,4197.045503,0.0
min,1109.000000,21995.000000,1.0
25%,1120.000000,22995.000000,1.0
50%,1149.000000,22995.000000,1.0
75%,1196.500000,23496.000000,1.0
max,1428.000000,33994.000000,1.0
input,1735.000000,29994.000000,1.0
ref_max_modified_z_score,4.952200,7418.825500,0.0


In [28]:
# to get the full feature list, just pass features=[] to the same function
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(refjobs, '629322', {"op": "ncatted", "op_instance": "3", "op_sequence": "32"}, features=[])
ret

True

In [29]:
df_rca

,majflt,read_bytes,inblock,timeslices,vol_ctxsw,rdtsc_duration,minflt,duration,systemtime,user+system,...,processor,cancelled_write_bytes,guest_time,write_bytes,delayacct_blkio_time,wchar,rssmax,outblock,num_procs,numtids
count,7.0,7.0,7.0,7.000000,7.000000,7.000000e+00,7.000000,7.000000,7.000000,7.000000,...,7.0,7.0,7.0,7.0,7.0,7.0,7.000000,7.0,7.0,7.0
mean,0.0,0.0,0.0,22.142857,17.714286,4.033804e+06,2524.857143,1188.428571,7141.142857,24566.571429,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7984.571429,8.0,1.0,1.0
std,0.0,0.0,0.0,1.345185,1.889822,2.735115e+05,27.431299,114.402589,1463.476048,4197.045503,...,0.0,0.0,0.0,0.0,0.0,0.0,36.836835,0.0,0.0,0.0
min,0.0,0.0,0.0,21.000000,17.000000,3.813848e+06,2508.000000,1109.000000,4999.000000,21995.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7968.000000,8.0,1.0,1.0
25%,0.0,0.0,0.0,21.500000,17.000000,3.854039e+06,2509.000000,1120.000000,6498.500000,22995.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7970.000000,8.0,1.0,1.0
50%,0.0,0.0,0.0,22.000000,17.000000,3.938599e+06,2509.000000,1149.000000,6998.000000,22995.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7972.000000,8.0,1.0,1.0
75%,0.0,0.0,0.0,22.000000,17.000000,4.113005e+06,2533.500000,1196.500000,7998.000000,23496.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7972.000000,8.0,1.0,1.0
max,0.0,0.0,0.0,25.000000,22.000000,4.550094e+06,2572.000000,1428.000000,8998.000000,33994.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,8068.000000,8.0,1.0,1.0
input,1.0,49152.0,96.0,140.000000,136.000000,5.978859e+06,2644.000000,1735.000000,9998.000000,29994.000000,...,0.0,0.0,0.0,4096.0,0.0,24681.0,7972.000000,8.0,1.0,1.0
ref_max_modified_z_score,0.0,0.0,0.0,2.832900,4.721500,3.583700e+00,42.493500,4.952200,1.350400,7418.825500,...,0.0,0.0,0.0,0.0,0.0,0.0,4.358300,0.0,0.0,0.0


## <a name="notes">Notes and Suggestions</a>

 * When we do `detect_outlier_ops` we might also want to know the top jobs (ranked) by tag.
   This would help for subsequent RCA analysis.
 * The columns of `df` and `scores_df` should be ordered in the same order as `sorted_features`.
